<a href="https://colab.research.google.com/github/roni762583/NEAT/blob/main/evolve_feedforward_spiking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# from: https://github.com/CodeReclaimers/neat-python/blob/master/examples/xor/evolve-feedforward.py

!pip install neat-python
!pip install visualize==0.5.1

import neat

# import local config file
from google.colab import files
uploaded = files.upload() 
config_file = list(uploaded.keys())[0]


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving config-spiking to config-spiking


In [10]:
# visualize.py
# from: https://github.com/CodeReclaimers/neat-python/blob/master/examples/xor/visualize.py

import warnings

import graphviz
import matplotlib.pyplot as plt
import numpy as np

class Visualize:

    def __init__(self):
        pass


    def plot_stats(statistics, ylog=False, view=False, filename='avg_fitness.svg'):
        """ Plots the population's average and best fitness. """
        if plt is None:
            warnings.warn("This display is not available due to a missing optional dependency (matplotlib)")
            return

        generation = range(len(statistics.most_fit_genomes))
        best_fitness = [c.fitness for c in statistics.most_fit_genomes]
        avg_fitness = np.array(statistics.get_fitness_mean())
        stdev_fitness = np.array(statistics.get_fitness_stdev())

        plt.plot(generation, avg_fitness, 'b-', label="average")
        plt.plot(generation, avg_fitness - stdev_fitness, 'g-.', label="-1 sd")
        plt.plot(generation, avg_fitness + stdev_fitness, 'g-.', label="+1 sd")
        plt.plot(generation, best_fitness, 'r-', label="best")

        plt.title("Population's average and best fitness")
        plt.xlabel("Generations")
        plt.ylabel("Fitness")
        plt.grid()
        plt.legend(loc="best")
        if ylog:
            plt.gca().set_yscale('symlog')

        plt.savefig(filename)
        if view:
            plt.show()

        plt.close()


    def plot_spikes(spikes, view=False, filename=None, title=None):
        """ Plots the trains for a single spiking neuron. """
        t_values = [t for t, I, v, u, f in spikes]
        v_values = [v for t, I, v, u, f in spikes]
        u_values = [u for t, I, v, u, f in spikes]
        I_values = [I for t, I, v, u, f in spikes]
        f_values = [f for t, I, v, u, f in spikes]

        fig = plt.figure()
        plt.subplot(4, 1, 1)
        plt.ylabel("Potential (mv)")
        plt.xlabel("Time (in ms)")
        plt.grid()
        plt.plot(t_values, v_values, "g-")

        if title is None:
            plt.title("Izhikevich's spiking neuron model")
        else:
            plt.title("Izhikevich's spiking neuron model ({0!s})".format(title))

        plt.subplot(4, 1, 2)
        plt.ylabel("Fired")
        plt.xlabel("Time (in ms)")
        plt.grid()
        plt.plot(t_values, f_values, "r-")

        plt.subplot(4, 1, 3)
        plt.ylabel("Recovery (u)")
        plt.xlabel("Time (in ms)")
        plt.grid()
        plt.plot(t_values, u_values, "r-")

        plt.subplot(4, 1, 4)
        plt.ylabel("Current (I)")
        plt.xlabel("Time (in ms)")
        plt.grid()
        plt.plot(t_values, I_values, "r-o")

        if filename is not None:
            plt.savefig(filename)

        if view:
            plt.show()
            plt.close()
            fig = None

        return fig


    def plot_species(statistics, view=False, filename='speciation.svg'):
        """ Visualizes speciation throughout evolution. """
        if plt is None:
            warnings.warn("This display is not available due to a missing optional dependency (matplotlib)")
            return

        species_sizes = statistics.get_species_sizes()
        num_generations = len(species_sizes)
        curves = np.array(species_sizes).T

        fig, ax = plt.subplots()
        ax.stackplot(range(num_generations), *curves)

        plt.title("Speciation")
        plt.ylabel("Size per Species")
        plt.xlabel("Generations")

        plt.savefig(filename)

        if view:
            plt.show()

        plt.close()


    def draw_net(config, genome, view=False, filename=None, node_names=None, show_disabled=True, prune_unused=False,
                node_colors=None, fmt='svg'):
        """ Receives a genome and draws a neural network with arbitrary topology. """
        # Attributes for network nodes.
        if graphviz is None:
            warnings.warn("This display is not available due to a missing optional dependency (graphviz)")
            return

        # If requested, use a copy of the genome which omits all components that won't affect the output.
        if prune_unused:
            genome = genome.get_pruned_copy(config.genome_config)

        if node_names is None:
            node_names = {}

        assert type(node_names) is dict

        if node_colors is None:
            node_colors = {}

        assert type(node_colors) is dict

        node_attrs = {
            'shape': 'circle',
            'fontsize': '9',
            'height': '0.2',
            'width': '0.2'}

        dot = graphviz.Digraph(format=fmt, node_attr=node_attrs)

        inputs = set()
        for k in config.genome_config.input_keys:
            inputs.add(k)
            name = node_names.get(k, str(k))
            input_attrs = {'style': 'filled', 'shape': 'box', 'fillcolor': node_colors.get(k, 'lightgray')}
            dot.node(name, _attributes=input_attrs)

        outputs = set()
        for k in config.genome_config.output_keys:
            outputs.add(k)
            name = node_names.get(k, str(k))
            node_attrs = {'style': 'filled', 'fillcolor': node_colors.get(k, 'lightblue')}

            dot.node(name, _attributes=node_attrs)

        used_nodes = set(genome.nodes.keys())
        for n in used_nodes:
            if n in inputs or n in outputs:
                continue

            attrs = {'style': 'filled',
                    'fillcolor': node_colors.get(n, 'white')}
            dot.node(str(n), _attributes=attrs)

        for cg in genome.connections.values():
            if cg.enabled or show_disabled:
                # if cg.input not in used_nodes or cg.output not in used_nodes:
                #    continue
                input, output = cg.key
                a = node_names.get(input, str(input))
                b = node_names.get(output, str(output))
                style = 'solid' if cg.enabled else 'dotted'
                color = 'green' if cg.weight > 0 else 'red'
                width = str(0.1 + abs(cg.weight / 5.0))
                dot.edge(a, b, _attributes={'style': style, 'color': color, 'penwidth': width})

        dot.render(filename, view=view)

        return dot


In [11]:
#
# from: 

"""
2-input XOR example -- this is most likely the simplest possible example.
"""

import os

import neat
import visualize

visualize = Visualize()

# 2-input XOR inputs and expected outputs.
xor_inputs = [(0.0, 0.0), (0.0, 1.0), (1.0, 0.0), (1.0, 1.0)]
xor_outputs = [(0.0,), (1.0,), (1.0,), (0.0,)]

config_file = 'config-feedforward'

def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        genome.fitness = 4.0
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        for xi, xo in zip(xor_inputs, xor_outputs):
            output = net.activate(xi)
            genome.fitness -= (output[0] - xo[0]) ** 2


def run(config_file):
    # Load configuration.
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5))

    # Run for up to 300 generations.
    winner = p.run(eval_genomes, 300)

    # Display the winning genome.
    print('\nBest genome:\n{!s}'.format(winner))

    # Show output of the most fit genome against training data.
    print('\nOutput:')
    winner_net = neat.nn.FeedForwardNetwork.create(winner, config)
    for xi, xo in zip(xor_inputs, xor_outputs):
        output = winner_net.activate(xi)
        print("input {!r}, expected output {!r}, got {!r}".format(xi, xo, output))

    node_names = {-1: 'A', -2: 'B', 0: 'A XOR B'}
    visualize.draw_net(config, winner, True, node_names=node_names)
    visualize.draw_net(config, winner, True, node_names=node_names, prune_unused=True)
    visualize.plot_stats(stats, ylog=False, view=True)
    visualize.plot_species(stats, view=True)

    p = neat.Checkpointer.restore_checkpoint('neat-checkpoint-4')
    p.run(eval_genomes, 10)


if __name__ == '__main__':
    # Determine path to configuration file. This path manipulation is
    # here so that the script will run successfully regardless of the
    # current working directory.
    #local_dir = os.path.dirname(__file__)
    #config_path = os.path.join(local_dir, 'config-feedforward')
    run('config-feedforward')



 ****** Running generation 0 ****** 

Population's average fitness: 2.25579 stdev: 0.36941
Best fitness: 2.99678 - size: (1, 2) - species 1 - id 14
Average adjusted fitness: 0.609
Mean genetic distance 1.658, standard deviation 0.423
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0   150      3.0    0.609     0
Total extinctions: 0
Generation time: 0.020 sec

 ****** Running generation 1 ****** 

Population's average fitness: 2.35749 stdev: 0.32894
Best fitness: 2.99980 - size: (2, 2) - species 1 - id 293
Average adjusted fitness: 0.562
Mean genetic distance 1.901, standard deviation 0.677
Population of 150 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1   146      3.0    0.562     0
     2    0     4       --       --     0
Total extinctions: 0
Generation time: 0.021 sec (0.020 average)

 ****** Running generation 2 ****** 

Popu

AttributeError: ignored

In [14]:
""" 2-input XOR example using Izhikevich's spiking neuron model. """

import multiprocessing
import os

from matplotlib import patches
from matplotlib import pylab as plt

import neat
import visualize

# Network inputs and expected outputs.
xor_inputs = ((0, 0), (0, 1), (1, 0), (1, 1))
xor_outputs = (0, 1, 1, 0)

# Maximum amount of simulated time (in milliseconds) to wait for the network to produce an output.
max_time_msec = 20.0


def compute_output(t0, t1):
    """Compute the network's output based on the "time to first spike" of the two output neurons."""
    if t0 is None or t1 is None:
        # If neither of the output neurons fired within the allotted time,
        # give a response which produces a large error.
        return -1.0
    else:
        # If the output neurons fire within 1.0 milliseconds of each other,
        # the output is 1, and if they fire more than 11 milliseconds apart,
        # the output is 0, with linear interpolation between 1 and 11 milliseconds.
        response = 1.1 - 0.1 * abs(t0 - t1)
        return max(0.0, min(1.0, response))


def simulate(genome, config):
    # Create a network of "fast spiking" Izhikevich neurons.
    net = neat.iznn.IZNN.create(genome, config)
    dt = net.get_time_step_msec()
    sum_square_error = 0.0
    simulated = []
    for idata, odata in zip(xor_inputs, xor_outputs):
        neuron_data = {}
        for i, n in net.neurons.items():
            neuron_data[i] = []

        # Reset the network, apply the XOR inputs, and run for the maximum allowed time.
        net.reset()
        net.set_inputs(idata)
        t0 = None
        t1 = None
        v0 = None
        v1 = None
        num_steps = int(max_time_msec / dt)
        net.set_inputs(idata)
        for j in range(num_steps):
            t = dt * j
            output = net.advance(dt)

            # Capture the time and neuron membrane potential for later use if desired.
            for i, n in net.neurons.items():
                neuron_data[i].append((t, n.current, n.v, n.u, n.fired))

            # Remember time and value of the first output spikes from each neuron.
            if t0 is None and output[0] > 0:
                t0, I0, v0, u0, f0 = neuron_data[net.outputs[0]][-2]

            if t1 is None and output[1] > 0:
                t1, I1, v1, u1, f0 = neuron_data[net.outputs[1]][-2]

        response = compute_output(t0, t1)
        sum_square_error += (response - odata) ** 2

        # print(genome)
        # visualize.plot_spikes(neuron_data[net.outputs[0]], False)
        # visualize.plot_spikes(neuron_data[net.outputs[1]], True)

        simulated.append((idata, odata, t0, t1, v0, v1, neuron_data))

    return sum_square_error, simulated


def eval_genome(genome, config):
    sum_square_error, simulated = simulate(genome, config)
    return 10.0 - sum_square_error


def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        genome.fitness = eval_genome(genome, config)


def run(config_path):
    # Load the config file, which is assumed to live in
    # the same directory as this script.
    config = neat.Config(neat.iznn.IZGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_path)

    # For this network, we use two output neurons and use the difference between
    # the "time to first spike" to determine the network response.  There are
    # probably a great many different choices one could make for an output encoding,
    # and this choice may not be the best for tackling a real problem.
    config.output_nodes = 2

    pop = neat.population.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    pop.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    pop.add_reporter(stats)

    pe = neat.ParallelEvaluator(multiprocessing.cpu_count(), eval_genome)
    winner = pop.run(pe.evaluate, 3000)

    # Display the winning genome.
    print('\nBest genome:\n{!s}'.format(winner))

    node_names = {-1: 'A', -2: 'B'}
    visualize.draw_net(config, winner, True, node_names=node_names)
    visualize.draw_net(config, winner, True, node_names=node_names, prune_unused=True)
    visualize.plot_stats(stats, ylog=False, view=True)
    visualize.plot_species(stats, view=True)

    # Show output of the most fit genome against training data, and create
    # a plot of the traces out to the max time for each set of inputs.
    print('\nBest network output:')
    plt.figure(figsize=(12, 12))
    sum_square_error, simulated = simulate(winner, config)
    for r, (inputData, outputData, t0, t1, v0, v1, neuron_data) in enumerate(simulated):
        response = compute_output(t0, t1)
        print("{0!r} expected {1:.3f} got {2:.3f}".format(inputData, outputData, response))

        axes = plt.subplot(4, 1, r + 1)
        plt.title("Traces for XOR input {{{0:.1f}, {1:.1f}}}".format(*inputData), fontsize=12)
        for i, s in neuron_data.items():
            if i in [0, 1]:
                t, I, v, u, fired = zip(*s)
                plt.plot(t, v, "-", label="neuron {0:d}".format(i))

        # Circle the first peak of each output.
        circle0 = patches.Ellipse((t0, v0), 1.0, 10.0, color='r', fill=False)
        circle1 = patches.Ellipse((t1, v1), 1.0, 10.0, color='r', fill=False)
        axes.add_artist(circle0)
        axes.add_artist(circle1)

        plt.ylabel("Potential (mv)", fontsize=10)
        plt.ylim(-100, 50)
        plt.tick_params(labelsize=8)
        plt.grid()

    plt.xlabel("Time (in ms)", fontsize=10)
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
    plt.savefig("traces.png", dpi=90)
    plt.show()


if __name__ == '__main__':
    #local_dir = os.path.dirname(__file__)
    run('config-spiking')

Population's average fitness: 7.82380 stdev: 0.91865
Best fitness: 8.37590 - size: (2, 6) - species 17 - id 3153

Species 104 with 2 members is stagnated: removing it

Species 100 with 2 members is stagnated: removing it

Species 105 with 3 members is stagnated: removing it

Species 102 with 2 members is stagnated: removing it

Species 103 with 2 members is stagnated: removing it
Average adjusted fitness: 0.930
Mean genetic distance 10.018, standard deviation 3.254
Population of 503 members in 193 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     4   58     5      8.1    0.967    11
     6   58     3      8.1    0.965     8
    10   58     4      8.0    0.956    12
    13   58     3      8.1    0.964    17
    14   58     3      8.0    0.955    33
    15   58     2      8.1    0.963    41
    17   58     2      8.2    0.974    23
    18   58     2      8.0    0.955    33
    22   58     3      8.1    0.962    31
    25   58     3      8.0

Process ForkPoolWorker-2:
Process ForkPoolWorker-1:


Population's average fitness: 7.85057 stdev: 0.86329
Best fitness: 8.37590 - size: (2, 6) - species 17 - id 3153

Species 44 with 5 members is stagnated: removing it

Species 108 with 5 members is stagnated: removing it

Species 109 with 2 members is stagnated: removing it

Species 110 with 2 members is stagnated: removing it

Species 112 with 3 members is stagnated: removing it

Species 40 with 3 members is stagnated: removing it

Species 39 with 4 members is stagnated: removing it
Average adjusted fitness: 0.933


Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 352, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
 

KeyboardInterrupt: ignored